In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import stopwords
from termcolor import colored
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix
from sklearn.pipeline import Pipeline
from termcolor import colored
#from sklearn.metrics import f1_score,roc_auc_score,
#from sklearn.metrics import recall_score, precision_score,accuracy_score

In [4]:
# Setting stopwords
#STOPWORDS = set(stopwords.words('english'))
from nltk.stem.wordnet import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
STOPWORDS=set(stopwords.words('english')+list(punctuation))
STOPWORDS.remove("not")

In [5]:
Tweets=pd.read_csv('data/Sandy20102.csv')

In [6]:
Tweets.rename(columns={'tweet id': 'TweetsID', 'tweet': 'TweetsText','label': 'Status'}, inplace=True)

In [7]:
print("Total tweets in the dataset: {}".format(Tweets.shape[0]))
Tweet_count = Tweets['Status'].value_counts()
Tweet_count

Total tweets in the dataset: 10008


on-topic     6138
off-topic    3870
Name: Status, dtype: int64

In [ ]:
Tweets.info()

In [ ]:
Tweets.head()

In [8]:
Tweets['TweetsID']=Tweets['TweetsID'].str.strip(' "\'')

In [5]:
Tweets.head()

,TweetsID,TweetsText,Status
0,'262596552399396864',I've got enough candles to supply a Mexican fa...,off-topic
1,'263044104500420609',Sandy be soooo mad that she be shattering our ...,on-topic
2,'263309629973491712',@ibexgirl thankfully Hurricane Waugh played it...,off-topic
3,'263422851133079552',@taos you never got that magnificent case of B...,off-topic
4,'262404311223504896',"I'm at Mad River Bar &amp; Grille (New York, N...",off-topic


In [9]:
Tweets[Tweets.duplicated(['TweetsID'], keep=False)]

,TweetsID,TweetsText,Status


In [13]:
Tweets[Tweets.duplicated(['TweetsText'], keep=False)]

,TweetsID,TweetsText,Status
5003,263360487566229504,"RT @HurricaneSandyw: FOR EVERY 100 RETWEETS, W...",on-topic
5012,263337011342106624,RT @isardasorensen: Rainbow over today's #NYC ...,on-topic
5015,263031793505144832,RT @MikeDrucker: I think he's busy being the f...,on-topic
5026,262612009718276097,RT @XHurricaneSandy: I went to highschool wit ...,on-topic
5028,262598022834298880,RT @BigFrankenStorm: I WENT TO HIGHSCHOOL WIT ...,on-topic
...,...,...,...
9969,262410768379678721,RT @aSandyHurricane: BLOWIN SO MANY NIGGAS THE...,on-topic
9975,263114082675212288,RT @FillWerrell: Snooki is a lot like Hurrican...,on-topic
9982,262561573980209152,RT @BigFrankenStorm: I WENT TO HIGHSCHOOL WIT ...,on-topic
9993,263008717027549184,RT @SandysHurricane: DIS BITCH WAS LIKE I'M DY...,on-topic


In [18]:
distinct_Tweets=Tweets[Tweets['TweetsText'].str.startswith('RT')!= True]

In [19]:
duplicate_Tweets=Tweets[Tweets.duplicated(['TweetsText'], keep=False)]

In [20]:
distinct_Tweets[distinct_Tweets.duplicated(['TweetsText'], keep=False)]

,TweetsID,TweetsText,Status
5534,263043704586125312,@justinbieber: everyone dealing with the hurri...,on-topic
6535,263035810885033984,@justinbieber: everyone dealing with the hurri...,on-topic


In [21]:
final_distinct_Tweets=distinct_Tweets[distinct_Tweets['TweetsID'].str.contains('263035810885033984')!= True]

In [22]:
final_distinct_Tweets[final_distinct_Tweets.duplicated(['TweetsText'], keep=False)]

,TweetsID,TweetsText,Status


In [ ]:
print("Total distinct tweets in the dataset: {}".format(final_distinct_Tweets.shape[0]))
Tweet_count = final_distinct_Tweets['Status'].value_counts()
Tweet_count

In [ ]:
final_distinct_Tweets.info()

In [ ]:
final_distinct_Tweets.isnull().any()

In [ ]:
final_distinct_Tweets.head()

In [ ]:
final_distinct_Tweets.tail()

In [ ]:
print(final_distinct_Tweets.shape)

In [ ]:
Tweet_count = final_distinct_Tweets['Status'].value_counts()
Tweet_count

In [14]:
mydata=final_distinct_Tweets.copy()

In [ ]:
mydata.to_csv('data/distinct_data.csv', index = False)
duplicate_Tweets.to_csv('data/duplicate_Tweets.csv', index = False)

plt.figure(figsize = (12, 8))
Tweets['Status'].value_counts().plot.bar(['ON-Topic', 'OFF-Topic'], Tweet_count.values, color = ['g', 'r'])
for i, v in enumerate(Tweet_count.values):
    plt.text(i - 0.1, v + 100, str(v))
    plt.xlabel("Tweets Categories")
plt.ylabel("Tweets count")
plt.title("Visual Representation of Data")
#plt.legend()

In [ ]:
plt.figure(figsize = (7,9))
plt.bar(['ON-Topic', 'OFF-Topic'], Tweet_count.values, color = ['g', 'r'])
for i, v in enumerate(Tweet_count.values):
    plt.text(i - 0.1, v + 100, str(v))
plt.xlabel("Tweets Categories")
plt.ylabel("Tweets count")
plt.title("CLASS DISTRIBUTION: ON-Topic vs OFF-Topic")

In [23]:
#Collecting the hashtags

def hashtag_extract(x):
    hashtags = []
    
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    return hashtags
#tweet = re.sub(r'#(\S+)', r' \1 ', tweet)

In [24]:
# extracting hashtags from relevant tweets
HT_On_Topic = hashtag_extract(final_distinct_Tweets['TweetsText'][final_distinct_Tweets['Status'] =='on-topic'])

# extracting hashtags from non-relevant tweets
HT_Off_Topic = hashtag_extract(final_distinct_Tweets['TweetsText'][final_distinct_Tweets['Status'] =='off-topic'])

#  list
HT_On_Topic = sum(HT_On_Topic,[])
HT_Off_Topic = sum(HT_Off_Topic,[])

In [32]:
#HT_On_Topic

In [26]:
HT_On_Topic = [x.lower() for x in HT_On_Topic if len(x)>1]
HT_Off_Topic = [x.lower() for x in HT_Off_Topic if len(x)>1]

In [28]:
from collections import Counter
count_HT_On_Topic_hashtags = dict(Counter(HT_On_Topic))
count_HT_Off_Topic_hashtags = dict(Counter(HT_Off_Topic))

In [33]:
#count_HT_On_Topic_hashtags

In [30]:
Dframe_HT_On_Topic_hashtags = pd.DataFrame({'Hashtag': list(count_HT_On_Topic_hashtags.keys()),
                  'Count': list(count_HT_On_Topic_hashtags.values())})

In [31]:
Dframe_HT_On_Topic_hashtags.head()

,Hashtag,Count
0,hurricanesandy,77
1,sandy,472
2,godisgood,1
3,hurricane,109
4,nyc,44


In [ ]:
# selecting top 10 most frequent hashtags     
Dframe_HT_On_Topic_hashtags = Dframe_HT_On_Topic_hashtags.nlargest(columns="Count", n=10) 
plt.figure(figsize=(20,30))
ax = sns.barplot(data=Dframe_HT_On_Topic_hashtags, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
ax.set(title = 'Visual Representation of Hashtags from Relevant Tweets')

In [ ]:
Dframe_HT_Off_Topic_hashtags = pd.DataFrame({'Hashtag': list(count_HT_Off_Topic_hashtags.keys()),
                  'Count': list(count_HT_Off_Topic_hashtags.values())})

In [ ]:
# selecting top 10 most frequent hashtags     
Dframe_HT_Off_Topic_hashtags = Dframe_HT_Off_Topic_hashtags.nlargest(columns="Count", n=10) 
plt.figure(figsize=(20,30))
ax = sns.barplot(data=Dframe_HT_Off_Topic_hashtags, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
ax.set(title = 'Visual Representation of Hashtags from Non-relevant Tweets')

In [ ]:
#mydayta.tail(500)

In [ ]:
#mydayta.TweetsID=='262970751861260288',mydayta[mydayta['TweetsText']]

In [ ]:
#count_HT_Off_Topic_hashtags

In [ ]:
# Define processing methods
#wordNetLemmatizer = WordNetLemmatizer()
#porterStemmer = PorterStemmer()

In [15]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

In [16]:
def process_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', 'RT', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()
    return ' '.join(words)

In [17]:
mydata['TweetsText']=mydata['TweetsText'].apply(process_tweet)

In [18]:
mydata.head()

,TweetsID,TweetsText,Status
0,'262596552399396864',i've got enough candles to supply a mexican fa...,off-topic
1,'263044104500420609',sandy be soooo mad that she be shattering our ...,on-topic
2,'263309629973491712',USER_MENTION thankfully hurricane waugh played...,off-topic
3,'263422851133079552',USER_MENTION you never got that magnificent ca...,off-topic
4,'262404311223504896',"i'm at mad river bar &amp; grille (new york, n...",off-topic


In [ ]:
#Encode Target Data

In [19]:
mydata.loc[mydata['Status']=='off-topic','Status']=0
mydata.loc[mydata['Status']=='on-topic','Status']=1

In [20]:
print("Total distinct tweets in the dataset: {}".format(final_distinct_Tweets.shape[0]))
Tweet_count = final_distinct_Tweets['Status'].value_counts()
Tweet_count

Total distinct tweets in the dataset: 7466


off-topic    3773
on-topic     3693
Name: Status, dtype: int64

In [ ]:
print("Total distinct tweets in the dataset: {}".format(mydata.shape[0]))
Tweet_count = mydata['Status'].value_counts()
Tweet_count

In [ ]:
#Extract On-Topic Tweets for Sentiment Analysis

In [ ]:
ontopic2Dataframe=pd.DataFrame(mydata[mydata['Status']==1])
ontopic2Dataframe.head()

Loading On-Topic Tweets for Sentiment Analysis
SentimentTweets=pd.read_csv('data/onTopic_Tweets+results2.txt',sep='\t')
SentimentTweets['Final']=(SentimentTweets['Negative'])+(SentimentTweets['Positive']).astype('int')
#Save On_Topic Tweets to CSV file
#ontopic2Dataframe.drop(['TweetsID','Status'], axis = 1, inplace = True)
ontopic2Dataframe.to_csv('data/onTopic_Tweets.txt', index = False)

Drop Columns not needed for Predictions

In [21]:
mydata.drop(['TweetsID'], axis = 1, inplace = True)

In [22]:
dF_features=mydata['TweetsText']
dF_target=mydata['Status']

In [23]:
X_train, x_test, y_train, y_test = train_test_split(dF_features, dF_target, test_size=0.2, random_state=11)

In [24]:
train_dataset = pd.DataFrame({'TweetsText': X_train,'Category': y_train})
print(train_dataset['Category'].value_counts())

0    2989
1    2983
Name: Category, dtype: int64


In [ ]:
len(train_dataset)/len(mydata)*100

In [ ]:
#train_dataset

In [25]:
test_dataset = pd.DataFrame({'TweetsText': x_test,'Category': y_test})
print(test_dataset['Category'].value_counts())

0    784
1    710
Name: Category, dtype: int64


In [ ]:
len(test_dataset)/len(mydata)*100

In [26]:
def lemmatize_func(tweets):
    tweets_Text=tweets.lower()
    words = word_tokenize(tweets_Text)
    valid_Words=[]
    for word in words :
        if word in STOPWORDS:continue
        valid_Words.append(word)
    return [lemma.lemmatize(word) for word in valid_Words]

In [ ]:
TF-IDF

In [27]:
cV=TfidfVectorizer(analyzer=lemmatize_func,min_df=1,stop_words=STOPWORDS)

In [28]:
train_tweet_vector = cV.fit_transform(train_dataset['TweetsText'])
test_tweet_vector = cV.transform(test_dataset['TweetsText'])

In [ ]:
Train the Classifiers

In [40]:
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [ ]:
estimators = {
    'MultinomialNB': MultinomialNB(),
     'DecisionTree': DecisionTreeClassifier()}

In [30]:
def validation_Report():
    for estimator_name, estimator_object in estimators.items():
        kfold = KFold(n_splits=10, random_state=7, shuffle=True)
        scores = cross_val_score(estimator=estimator_object,
        X=train_tweet_vector, y=y_train, cv=kfold)
        print(f'{estimator_name:>20}: ' +
        f'mean accuracy={scores.mean():.2%}; ' +
        f'standard deviation={scores.std():.2%}')

In [ ]:
def estimator(model,X,y):
    clf = model
    clf.fit(train_tweet_vector, y_train)
    y_pred = clf.predict(test_tweet_vector)
    return y_pred

In [ ]:
predicted=predictionsList()

In [ ]:
predicted_MNB=predicted[0]

In [ ]:
predicted_MNB

In [ ]:
predicted_DT=predicted[1]

In [ ]:
predicted_DT

In [ ]:
DT=estimator(DecisionTreeClassifier(),train_tweet_vector,y_train)


In [ ]:
DT

In [ ]:
accuracy_Report()

In [32]:
MNB=MultinomialNB()

In [42]:
DT=DecisionTreeClassifier()

In [ ]:
y_train=y_train.astype('int')

In [35]:
MNB.fit(train_tweet_vector,y_train)

MultinomialNB()

In [41]:
print(f'{MNB.score(test_tweet_vector, y_test):.2%}')

79.72%


In [45]:
print(f'{DT.score(test_tweet_vector, y_test):.2%}')

89.09%


In [44]:
DT.fit(train_tweet_vector,y_train)

DecisionTreeClassifier()

In [ ]:
prediction = MNB.predict(test_tweet_vector)
prediction

In [ ]:
DTPred=DT.predict(test_tweet_vector)
DTPred

In [ ]:
actual=np.array(y_test)
actual

In [ ]:
count=0
for i in range(len(prediction)):
    if prediction[i]==actual[i]:
        count=count+1

In [ ]:
count1=0

In [ ]:
count1=0
for i in range(len(DTPred)):
    if DTPred[i]==actual[i]:
        count1=count1+1

In [ ]:
acc=count/len(prediction)
print("Accuracy: %.2f%%" % (acc * 100.0))

In [ ]:
acc1=count1/len(DTPred)
print("Accuracy: %.2f%%" % (acc1 * 100.0))

In [ ]:
#evaluate_results(actual, prediction)

In [ ]:
x_test.iloc[13],y_test.iloc[13]

In [ ]:
Pred

In [ ]:
DT=estimator(DecisionTreeClassifier(),train_tweet_vector,y_train)

In [ ]:
predicted_MNB=Pred[0]
predicted_DT=Pred[0]

In [ ]:
predicted_MNB


In [ ]:
predicted_DT

In [2]:
import utils

ModuleNotFoundError: No module named 'utils'

In [3]:
from python_utils import converters


ModuleNotFoundError: No module named 'python_utils'

In [5]:
import utils

ModuleNotFoundError: No module named 'utils'